<center><img src="img/logo_hse_black.jpg"></center>

<h1><center>Natural Language Processing</center></h1>
<h2><center>Week3 Seminar - embeddings</center></h2>

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

# Самые известные модели
## GloVe - Global VEctors

* Шаг 1. Собрать матрицу совстречаемости слов в корпусе (или матрицу PMI)
$$ PMI_{uv} = \frac{n_{vu}n}{n_v n_u} $$
* Шаг 2. Разложить эту матрицу на произведение двух других матриц

<center><img src='https://miro.medium.com/max/19360/1*yNFtSAwoht-nlfzW_Um_yw.png' width=600></center>

Как это можно сделать?

* Использовать типовые алгоритмы матричной факторизации (NMF, SVD..)
* Использовать градиентный спуск для какой-нибудь функции потерь
$$L = \sum\limits_{u,v} f(n_{uv})   (\phi_u^T \theta_v + b_u + \tilde{b}_v - \text{log}   n_{uv})^2$$

## Skip-Gram

"Слово предсказывает слова из своего контекста"
* Для каждого слова инициализируем вектор-ембеддинг
* Обновляем их таким образом, чтобы вектор текущего слова был похож на вектора окружения
$$ L = \frac{1}{T}\sum\limits_{t=1}^T\ \sum\limits_{-n \leq j \leq n , \neq 0} \text{log} \space p(w_{t+j} \: | \: w_t) $$
$$ p(w_{t+j} \: | \: w_t ) = \dfrac{\text{exp}({v_{w_t}^\top v'_{w_{t+j}}})}{\sum_{w_i \in V} \text{exp}({v_{w_t}^\top v'_{w_i}})} $$

<center><img src='http://ruder.io/content/images/2016/02/skip-gram.png'></center>

## CBOW - Continious Bag Of Words

"Комбинация слов контекста предсказывает текущее слово"
* Для каждого слова инициализируем вектор-ембеддинг
* Обновляем их таким образом, чтобы комбинация векторов контекста была наиболее похожа на текущее слово

$$L = \frac{1}{T}\sum\limits_{t=1}^T\ \text{log} \space p(w_t \: | \: w_{t-n} , \cdots , w_{t-1}, w_{t+1}, \cdots , w_{t+n})$$
$$ p(w_t \: |\: w_{t-n} , \cdots , w_{t-1}, w_{t+1}, \cdots , w_{t+n} ) = \dfrac{\text{exp}({v_{w_t}^\top u_t})}{\sum_{w_i \in V} \text{exp}({v_{w_t}^\top u_t})} $$

<center><img src='http://ruder.io/content/images/2016/02/cbow.png'></center>

Дополнительная информация - [туть](https://cs224d.stanford.edu/lecture_notes/notes1.pdf)

## Основные практические проблемы
* Out of vocabulary words
* Векторные представления н-грамм, предложений или текстов
* Применимость word2vec as is в моделях

In [ ]:
!mkdir data
!wget https://www.dropbox.com/s/jrhs1gxcql57r73/amazon_cells_labelled.txt?dl=0 -O ./data/amazon_cells_labelled.txt

In [ ]:
df_text = pd.read_csv('./data/amazon_cells_labelled.txt', sep='\t', names=['review', 'label'])

In [ ]:
df_text.head()

### Стандартная нормализация текстов без изысков

In [ ]:
import nltk
tokenizer = nltk.tokenize.WordPunctTokenizer()


In [ ]:
# Приводим к нижнему регистру
df_text.loc[:, 'review'] = df_text.review.str.lower()
# Токенизируем
df_text.loc[:, 'review'] = df_text.review.apply(lambda x: tokenizer.tokenize(x))

In [ ]:
X_texts = df_text.review.values
y = df_text.label.values

In [ ]:
# !pip install gensim

In [ ]:
# Если бы вы хотели обучать свой word2vec

from gensim.models import Word2Vec
# model = Word2Vec(your_corpus, 
#                  size=32,      
#                  min_count=5,  
#                  window=5).wv 

# Но мы используем уже предобученные

Список предобученных эмбедингов можно найти [тут](https://github.com/RaRe-Technologies/gensim-data#models)

In [ ]:
import gensim.downloader as api
model = api.load('glove-twitter-25')

In [ ]:
model['microphone']

In [ ]:
model.most_similar('microphone')

In [ ]:
model.most_similar(positive=['coder', 'money'], negative=['brain'])

In [ ]:
model.vector_size

In [ ]:
words = list(model.vocab.keys())[:30000]

print(words[::100])

In [ ]:
word_vectors = np.array([model[w] for w in words])
word_vectors.shape

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, n_jobs=4)
tsne.fit(word_vectors)

word_vectors_tsne = tsne.embedding_

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
draw_vectors(word_vectors_tsne[:, 0], word_vectors_tsne[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

In [ ]:
# Напишите функцию, которая на вход будет брать список слов, а на выходе будет возвращать усредненный вектор из эмбеддингов
# Если слова не было в словаре, то при расчете среднего оно не должно учитываться!

def mean_word_vectors(text, model_wv, emb_size):
    mean_vector = np.zeros(emb_size)
    
    ## Your code here
    
    return mean_vector

In [ ]:
X_embeddings = np.array([mean_word_vectors(t, model, model.vector_size) for t in X_texts])

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_embeddings ,y, test_size = 0.2, random_state = 123)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

model = LogisticRegression()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_hat = model.predict_proba(X_valid)

roc_auc_score(y_valid, y_hat[:, 1])